## Overhearing population
### https://iopscience.iop.org/article/10.1209/0295-5075/101/68004/meta
### https://langev.com/pdf/0bqCwMtiFdoJ.pdf

### 1. basic speaker-hearer rules

In [2]:
## basic speaker-hearer rules

import random

## s is one node of P (population-graph)
## M is the dictionary of memories {node:memory list of node}
## V is a predefined vocabulary
def speaker_rule(s,M,V):
    ## the speaker s selects one word w from her inventory
    if len(M[s])==0:
        w = random.choice(V)
        M[s]+=[w]
    else:
        w = random.choice(M[s])
    return w

## h is one node of P (population-graph)
## word is the word uttered by the speaker
## M is the dictionary of memories {node:memory list of node}
def hearer_rule(h,word,M):
    ## the hearer receives word 
    success = False
    if word in M[h]:
        success = True
    return success

### 2. dynamics

In [3]:
## s is one node of P (population-graph)
## h is one node of P (population-graph)
## word is speaker's word
## M is the dictionary of memories {node:memory list of node}
## V is a predefined vocabulary

def dynamics(s,h,word,M,V):
    ## hearer's state
    success = hearer_rule(h,word,M)
    ## alignment strategies
    if success == True:
        M[s]=[word]
        M[h]=[word]
    else:
        M[h]+=[word]
    return M  

In [10]:
import networkx as nx

T = 1000
number_nodes = 10
P = nx.complete_graph(number_nodes)
V = range(10)
M = {node:[] for node in P.nodes()}
percentage_over = 0.1

for t in range(T):
    ## speaker
    others = list(P.nodes())
    speaker = random.choice(others)
    ## hearer!=speaker!
    others.pop(speaker)
    hearer_central = random.choice(others)
    ## overhearers (percentage of the neighbors)
    overhearers = random.sample(list(dict(P[speaker]).keys()),int(percentage_over*len(P[speaker])))
    ## one overhearer to interact with the speaker
    word = speaker_rule(speaker,M,V)
    ## speaker-hearer interaction!
    M = dynamics(speaker,hearer_central,word,M,V)
    for hearer in overhearers:
        success = hearer_rule(hearer,word,M)
        if success == True:
            M[hearer]=[word]
        else:
            M[hearer]+=[word]     

In [11]:
M

{0: [0],
 1: [0],
 2: [0],
 3: [0],
 4: [0],
 5: [0],
 6: [0],
 7: [0],
 8: [0],
 9: [0]}